In [1]:
from import_library import *

In [2]:
data_type ='csv' #mat, csv, mix
# dataloader
clinical_data = pd.read_csv('./data/clinical data/clinical_data.csv')[['subj', 'AD8主觀認知障礙', 'MOCA客觀認知測驗分數', 'HADS_A焦慮程度', 'HADS_D憂鬱程度']]
raw_data_path = './data/image data/mdALFF matrix excel/' #raw_data_path = './data/image data/dFC matrix matlab/'
next_path ='mdALFF.csv' #next_path ='C0005FC_z.mat'
def loader_(data_path = None, data_type='mat' ,mat_dtype='FCM_mean'):
    if '.mat' in data_path:
        mat = scipy.io.loadmat(data_path)
        get_value = mat[mat_dtype ]
        return get_value.shape

    else:
        pd_tb = pd.read_csv(data_path)
        pd_patient_index = pd_tb[pd_tb.columns[0]]
        pd_patient_value = pd_tb[pd_tb.columns[1::]]
        return pd_patient_index, pd_patient_value
        
_, patient_value = loader_(data_path = os.path.join(raw_data_path,next_path))
patient_value_index = clinical_data['subj']

In [3]:
skf = StratifiedKFold(shuffle=True, n_splits=3, random_state=123) #random seed = 123

In [4]:
# model list
def model_list(grid=False, seed=123):
    # svr_param = {'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),'C' : [1,5,10],'degree' : [3,8],'coef0' : [0.01,10,0.5],'gamma' : ('auto','scale')}
    # SVR_ = SVR()
    
    xgbr_param = {'nthread':[4], 'objective':['reg:squarederror'], 'learning_rate': [.03, 0.05, .07], 'max_depth': [5, 6, 7], 'min_child_weight': [4], 
                                    'subsample': [0.7], 'colsample_bytree': [0.7], 'n_estimators': [200, 300, 400, 500]}
    XGBR_ =XGBRegressor(random_state=seed)

    lgbmr_param =  {'num_leaves': [7, 14, 21, 28, 31, 50], 'learning_rate': [0.001, 0.02, 0.05, 0.1, 0.15, 0.20, 0.25], 'max_depth': [15, 20, 25, 30, 35], 
                                        'n_estimators': [50, 100, 200, 500], 'cat_smooth': [1,10, 15, 20, 35]}
    LGBMR_ = LGBMRegressor(random_state=seed)

    mlpr_param =  {'hidden_layer_sizes': [(100,), (50,100,), (50,75,100,)], 'activation': ['relu', 'tanh', 'logistic', 'identity'], 'solver': ['adam', 'lbfgs'], 'learning_rate' : ['constant', 'adaptive', 'invscaling']}

    MLPR_ = MLPRegressor(hidden_layer_sizes =(50,100), activation = 'relu', solver = 'adam', learning_rate_init=0.0003, max_iter=800, random_state=seed)

    if grid ==True:
        # model_stack = {"SVR":[SVR_, svr_param], "XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
        model_stack = {"MLPR":[MLPR_, mlpr_param], "XBGR":[XGBR_, xgbr_param], "LGBMR": [LGBMR_, lgbmr_param]}
    else:
        # model_stack = {"SVR":SVR_, "XBGR":XGBR_, "LGBMR": LGBMR_}
        # model_stack = {"XBGR":XGBR_, "LGBMR": LGBMR_}
        model_stack = {"MLPR":MLPR_, "XBGR":XGBR_, "LGBMR": LGBMR_}
    return model_stack

In [5]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if True:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        outcome_name = 'MOCA客觀認知測驗分數' #AD8主觀認知障礙, MOCA客觀認知測驗分數
        label_ = clinical_data[outcome_name]
        

        for times_, (train_index, valid_index) in enumerate(skf.split(np.array(patient_value), np.array(label_))):
            model_stack = model_list()
            for model in model_stack:
                print("Outcome: {}".format(outcome_name))
                print("--Start Training {} model Times: [{}] --".format(model, times_+1))
                print("Data Length. Training : {} || Validation: {}".format(len(train_index), len(valid_index)))

                X_train, X_val, y_train, y_val = np.array(patient_value)[train_index], np.array(patient_value)[valid_index], np.array(label_)[train_index], np.array(label_)[valid_index]
                model_stack[model].fit(X_train, y_train)
                val_pred =  model_stack[model].predict(X_val)
                train_pred =  model_stack[model].predict(X_train)

                print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                print('Valid MAE: ', round(metrics.mean_absolute_error(y_val,val_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_val,val_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_val,val_pred),5))
                print('----------------------------------------------------')
            print('\n')
                

Outcome: MOCA客觀認知測驗分數
--Start Training MLPR model Times: [1] --
Data Length. Training : 56 || Validation: 29
Train MAE:  2.93999  ||  MSE:  12.52487  ||  MAPE:  0.12247
Valid MAE:  3.26696  ||  MSE:  15.61122  ||  MAPE:  0.14653
----------------------------------------------------
Outcome: MOCA客觀認知測驗分數
--Start Training XBGR model Times: [1] --
Data Length. Training : 56 || Validation: 29
Train MAE:  0.00022  ||  MSE:  0.0  ||  MAPE:  1e-05
Valid MAE:  3.76575  ||  MSE:  19.7413  ||  MAPE:  0.16364
----------------------------------------------------
Outcome: MOCA客觀認知測驗分數
--Start Training LGBMR model Times: [1] --
Data Length. Training : 56 || Validation: 29
Train MAE:  0.94765  ||  MSE:  1.27959  ||  MAPE:  0.03908
Valid MAE:  3.22751  ||  MSE:  14.31151  ||  MAPE:  0.14079
----------------------------------------------------


Outcome: MOCA客觀認知測驗分數
--Start Training MLPR model Times: [2] --
Data Length. Training : 57 || Validation: 28
Train MAE:  3.15177  ||  MSE:  14.14321  ||  MAPE: 

In [6]:
def fxn():
    warnings.warn("deprecated", DeprecationWarning)
if False:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        outcome_name = 'AD8主觀認知障礙' #AD8主觀認知障礙, MOCA客觀認知測驗分數
        label_ = clinical_data[outcome_name]
        for times_, (train_index, valid_index) in enumerate(skf.split(np.array(patient_value), np.array(label_))):
            model_stack = model_list(grid=True)
            for model in model_stack:
                print("Outcome: {}".format(outcome_name))
                print("--Start Training {} model Times: [{}] --".format(model, times_+1))
                print("Data Length. Training : {} || Validation: {}".format(len(train_index), len(valid_index)))
                X_train, X_val, y_train, y_val = np.array(patient_value)[train_index], np.array(patient_value)[valid_index], np.array(label_)[train_index].astype(np.float32), np.array(label_)[valid_index].astype(np.float32)

                model_grid = GridSearchCV(model_stack[model][0], model_stack[model][1], cv = 10, n_jobs = 5, verbose=True)
                
                model_grid.fit(X_train, y_train)
                print('Best parameter : [{}]\n'.format(model_grid.best_params_))
                val_pred =  model_grid.predict(X_val)
                train_pred =  model_grid.predict(X_train)
                # print([round(y-p, 3) for p, y in zip(train_pred, y_train)])
                
                # print(val_pred, y_val)
                print('Train MAE: ', round(metrics.mean_absolute_error(y_train,train_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_train,train_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_train,train_pred),5))
                print('Valid MAE: ', round(metrics.mean_absolute_error(y_val,val_pred), 5),' || ','MSE: ',round(metrics.mean_squared_error(y_val,val_pred),5),' || ','MAPE: ',round(metrics.mean_absolute_percentage_error(y_val,val_pred),5))
                print('----------------------------------------------------')
            print('\n')